In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
# Input Data Set #
Data ="Iris"
JobNameAbbrev = "IS"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,100)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Multiplier"],                                         # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0.2, 0.3],
                       "Type": ["Classification"],
                       "Partition": ["compute"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["03:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [3000]}                                                                # [1000, 30000, 100000]

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Random Forest Simulations

In [3]:
# ### Include Random Forest ###
# RandomForestParameterDictionary = {"Data":[Data],
#                        "Seed":list(range(0,100)),
#                        "TestProportion":[0.2],
#                        "CandidateProportion":[0.8],
#                        "SelectorType":["TreeEnsembleQBCFunction"],
#                        "ModelType":["RandomForestClassificationFunction"],
#                        "UniqueErrorsInput": [0],
#                        "n_estimators": [100], 
#                        "regularization": [0.01],
#                        "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
#                        "RashomonThreshold": [0],
#                        "Type": ["Classification"],
#                        "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
#                        "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
#                        "Memory": [1000]}                                                                # [1000, 30000, 100000]

# RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# # NOTE: Comment out chunk to not include random forest simulations. ###
# ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
# ParameterVector = ParameterVector.sort_values("Seed")
# ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [4]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["RashomonThresholdType"].astype(str) + 
    ParameterVector["RashomonThreshold"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_D", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_U", regex=True)
    .str.replace(r"Adder", "A", regex=True)
    .str.replace(r"Multiplier", "M", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True))

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

# Save Parameter Vector

In [5]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,RashomonThresholdType,RashomonThreshold,Type,Partition,Time,Memory,JobName,Output
0,Iris,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Multiplier,0.2,Classification,compute,03:59:00,3000,0IS_UM0.2,Iris/TreeFarms/Raw/0IS_UM0.2.pkl
1,Iris,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Multiplier,0.3,Classification,compute,03:59:00,3000,0IS_UM0.3,Iris/TreeFarms/Raw/0IS_UM0.3.pkl
2,Iris,1,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Multiplier,0.2,Classification,compute,03:59:00,3000,1IS_UM0.2,Iris/TreeFarms/Raw/1IS_UM0.2.pkl
3,Iris,1,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Multiplier,0.3,Classification,compute,03:59:00,3000,1IS_UM0.3,Iris/TreeFarms/Raw/1IS_UM0.3.pkl
4,Iris,2,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Multiplier,0.2,Classification,compute,03:59:00,3000,2IS_UM0.2,Iris/TreeFarms/Raw/2IS_UM0.2.pkl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Iris,97,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Multiplier,0.3,Classification,compute,03:59:00,3000,97IS_UM0.3,Iris/TreeFarms/Raw/97IS_UM0.3.pkl
196,Iris,98,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Multiplier,0.2,Classification,compute,03:59:00,3000,98IS_UM0.2,Iris/TreeFarms/Raw/98IS_UM0.2.pkl
197,Iris,98,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,Multiplier,0.3,Classification,compute,03:59:00,3000,98IS_UM0.3,Iris/TreeFarms/Raw/98IS_UM0.3.pkl
198,Iris,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,Multiplier,0.2,Classification,compute,03:59:00,3000,99IS_UM0.2,Iris/TreeFarms/Raw/99IS_UM0.2.pkl


In [6]:
# Save 
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)

---

In [7]:
# ### Simulations that failed ###
# FilterText = ['66IS_UNQ0.041', 
# '66IS_UNQ0.043', 
# '66IS_UNQ0.045', 
# '13IS_UNQ0.045', 
# '66IS_DPL0.041', 
# '66IS_DPL0.043', 
# '66IS_DPL0.045', 
# '13IS_DPL0.045']

# ### Rerun these simulations ###
# ParameterVector = ParameterVector[ParameterVector["JobName"].isin(FilterText)]
# ParameterVector.index = range(0,len(ParameterVector))
# ParameterVector
# ### Resave ###
# output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
# ParameterVector.to_csv(output_path, index=False)